<a href="https://colab.research.google.com/github/gabrielblins/palestra_gdg/blob/main/notebooks/GoogleExtended_OpenAILangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Integração OpenAI - Langchain

Vamos ao que interessa.

### Software base
Vamos instalar o langchain e algumas outras coisas

In [1]:
!pip install -U -q langchain unstructured pinecone-client openai tiktoken python-dotenv faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 108.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 105.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 137.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 104.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

# Integração com Pinecone

Para realizar uma integração completa usando exclusivamente o Maritaca, a API precisaria de um endpoint para captura dos embeddings, mas isso ainda não está disponível, então, a seguir iremos utilizar a OpenAI para criar as representações dos textos, enviar para o Pinecone e usar o Maritaca no final do fluxo. Funciona assim:

1. Capturamos os textos que queremos trabalhar, neste caso, a Portaria de Consolidação N 1 do Ministério da Saúde.

2. Vamos quebrar o texto em pequenas partes.

3. Interagir com a OpenAI e capturar os embeddings dos trechos criados.

4. Enviar esses trechos para o Pinecone.

Perceba que os passos acima são executados uma só vez ou sempre que houver necessidade de atualizar a estrutura. O resultado é uma base com diversos vetores que representam cada um dos trechos que criamos. O Pinecone, ou qualquer banco do mesmo tipo, viabiliza que sejam executadas consultas por similaridade. Dessa forma:

1. Agora que temos a base podemos enviar uma pergunta ou um texto qualquer, mas temos que criar a representação da pergunta e neste momento iremos chamar novamente a OpenAI para captura dos embeddings.

2. Em seguida, a Langchain interage com o Pinecone e pede para retornar os N vetores que mais se parecem com a pergunta/texto que enviamos.

Deste ponto em diante, já temos acesso aos N trechos semelhantes e, portanto, são os que possívelmente respondem a nossa pergunta. Vamos usar eles como contexto do prompt final com Maritaca.

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import OnlinePDFLoader

loader = OnlinePDFLoader("https://arxiv.org/pdf/2304.07880.pdf")

In [4]:
data = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [5]:
print (f'Você tem {len(data)} documento(s) na base')
print (f'Há {len(data[0].page_content)} caracteres no documento')

Você tem 1 documento(s) na base
Há 65146 caracteres no documento


In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=100, separators=['\n', ' ', ''])
texts = text_splitter.split_documents(data)

In [7]:
print (f'Agora há {len(texts)} documentos')

Agora há 192 documentos


In [8]:
texts[50].page_content

'Through our Portuguese pretraining, we observed that the improvement in NPM was higher in native datasets than that in translated datasets. For Sabi´a-65B, improvements over LLaMA-65B were mostly from the native subset. We hypothesize that this is due to the “mechanistic” nature of translated datasets: since they were translated from English, the baseline model already possesses the knowledge needed to solve them and gains little from learning the linguistic, syntactic, and grammatical knowledge of the'

### Configuração da OpenAI e Pinecone

Recomendo ler como o Pinecone funciona, há muita documentação pela internet.

O código a seguir realiza a integração entre Pinecone e OpenAI e para isso você vai precisar de chaves. Para manter as que uso de forma privada, criei um arquivo .env com o seguinte formato:

```
    OPENAI_API_KEY=key
    PINECONE_API_KEY=key
    PINECONE_API_ENV=key
    PINECONE_INDEX=key
```

Chamei de .env e coloquei na raiz do colab. Você pode fazer o mesmo para testar este notebook.

In [9]:
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone
import os

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [10]:
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']
PINECONE_API_ENV = os.environ['PINECONE_API_ENV']
INDEX_NAME = os.environ['PINECONE_INDEX']

In [11]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [12]:
# initialize pinecone
#pinecone.init(
#    api_key=PINECONE_API_KEY,
#    environment=PINECONE_API_ENV
#)
#index_name = INDEX_NAME

In [13]:
# Use o código a seguir para criar a base
#docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name, namespace=index_name)

# Use o código a seguir se já tiver um índice criado
# docsearch = Pinecone.from_existing_index(index_name=index_name, embedding=embeddings)

### Configuração da FAISS vector store

In [14]:
import faiss
from langchain.vectorstores import FAISS
import pickle

def store_embeddings(docs, embeddings, sotre_name, path):

    vectorStore = FAISS.from_documents(docs, embeddings)

    with open(f"{path}/faiss_{sotre_name}.pkl", "wb") as f:
        pickle.dump(vectorStore, f)

def load_embeddings(sotre_name, path):
    with open(f"{path}/faiss_{sotre_name}.pkl", "rb") as f:
        VectorStore = pickle.load(f)
    return VectorStore


In [15]:
docsearch = FAISS.from_texts([t.page_content for t in texts], embeddings)

### Base construída

Agora temos nossa base vetorizada! Nosso PDF contendo os dados da Portaria de Consolidação já está no Pinecone, vamos testar!

A variável query vai armazenar o texto que será usado para a pesquisa por similaridade. Note que ao criar o objeto docsearch a variável embeddings foi passada como parâmetro. Ela será usada para criar uma representação da query e comparar com os dados contidos no Pinecone. Até o momento, a Maritaca não tem essa opção de criar embeddings, então vamos usar a OpenAI mesmo.

In [16]:
query = "What are the Sabiá Models?" # faça uma pergunta ou escreva um texto.
docs = docsearch.similarity_search(query)

In [17]:
docs

[Document(page_content='We extended the training of three models — LLaMA 7B and 65B [67] as well as GPT-J [70] — originally trained on English-centric corpora, on Portuguese texts; these further pretrained models from LLaMA are denoted as Sabi´a, while the one derived from GPT-J is referred to as Sabi´a-J.3\n\n3.2 Sabi´a models', metadata={}),
 Document(page_content='We evaluate the Sabi´a models on the Portuguese Evaluation Tasks (Poeta) benchmark, which comprises 14 downstream NLP datasets in Portuguese: ASSIN 2 RTE and STS [51], ENEM Challenge [60], ENEM 2022 [40], FaQuAD [56], TweetSentBr [5], AG News [79], IMDB [35], MASSIVE [15], MKQA [33], BoolQ [11], SST2 [61], WSC [13], and BLUEX [1]. Half of them (ASSIN 2 RTE and STS, BLUEX, ENEM Challenge, ENEM 2022, FaQuAD, and TweetSentBr) were originally written in Portuguese, and the remaining ones were either manually', metadata={}),
 Document(page_content='Sabi´a: Portuguese Large Language Models\n\nRamon Pires∗, Hugo Abonizio∗, Thales

### Integração final

Agora vamos usar os textos vindos do Pinecone e pedir para nosso LLM, o MariTalk, criar uma resposta adequada.

In [18]:
from langchain.chat_models import ChatOpenAI
llm=ChatOpenAI(model_name="gpt-3.5-turbo")

In [19]:
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts.prompt import PromptTemplate

PROMPT = """
Use os trechos a seguir para responder a pergunta no final. Se você não souber a resposta, apenas diga que não sabe, não tente criar uma resposta.

{context}

Pergunta: {question}
Resposta:
"""

chain_prompt = PromptTemplate(input_variables=["context", "question"], template=PROMPT)

# note a variável llm sendo passada abaixo, ela é o Maritaca
chain = load_qa_chain(llm, chain_type="stuff", verbose=True, prompt=chain_prompt)

In [20]:
query = "What are the Sabiá Models?" # faça uma pergunta ou escreva um texto.

# no exemplo abaixo reduzimos o número de documentos para caber na janela do
# Maritaca
docs = docsearch.similarity_search(query, k=5)
chain.run(input_documents=docs, question=query)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use os trechos a seguir para responder a pergunta no final. Se você não souber a resposta, apenas diga que não sabe, não tente criar uma resposta.

We extended the training of three models — LLaMA 7B and 65B [67] as well as GPT-J [70] — originally trained on English-centric corpora, on Portuguese texts; these further pretrained models from LLaMA are denoted as Sabi´a, while the one derived from GPT-J is referred to as Sabi´a-J.3

3.2 Sabi´a models

We evaluate the Sabi´a models on the Portuguese Evaluation Tasks (Poeta) benchmark, which comprises 14 downstream NLP datasets in Portuguese: ASSIN 2 RTE and STS [51], ENEM Challenge [60], ENEM 2022 [40], FaQuAD [56], TweetSentBr [5], AG News [79], IMDB [35], MASSIVE [15], MKQA [33], BoolQ [11], SST2 [61], WSC [13], and BLUEX [1]. Half of them (ASSIN 2 RTE and STS, BLUEX, ENEM Challenge, ENEM 2022, FaQuAD, and TweetSentBr) were original

'The Sabiá models refer to the Portuguese Large Language Models that were trained on Portuguese texts using the LLaMA and GPT-J models as a starting point.'

As sacadas são movimentos simultâneos de ambos os olhos em duas ou mais fases para fixação na mesma direção. Elas podem ser usadas para estudar o comportamento ocular, os Movimentos Rápidos dos Olhos (REM) durante o estágio do sono, entre outras coisas.